In [1]:
import io
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression 

###########################################################################
#generate training/test data
########################################################################### 
bragging_file = io.open("Happy_clean.txt",'r',encoding="utf-8") 
bitching_file = io.open("FML_clean.txt",'r',encoding="utf-8") 


bragging_tweets = []
bitching_tweets = []

for _ in range(249000):
    bragging_tweets.append([bragging_file.readline().strip("\n"),0])
    bitching_tweets.append([bitching_file.readline().strip("\n"),1])
    
tweets = bragging_tweets+bitching_tweets
np.random.seed(42)
np.random.shuffle(tweets)


tweets_train = np.array(tweets[:348600])
tweets_test = np.array(tweets[348600:498000])

X_train = tweets_train[:,0]
Y_train = tweets_train[:,1]
X_test = tweets_test[:,0]
Y_test = tweets_test[:,1]

#########################################################################
#bag of words
#########################################################################

vectorizer = CountVectorizer()
X_train_counts = vectorizer.fit_transform(X_train)
X_test_counts = vectorizer.transform(X_test)


tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_test_tfidf = tfidf_transformer.transform(X_test_counts)


In [2]:
print (X_test[-1])

#Children 's #Day to u all.....may #GOD #bless and #protect our #children in the name of #JESUS #AMEN from... http:// fb.me/4ifqcgWnA


In [3]:

import keras.optimizers
from keras.models import Sequential 
from keras.layers import Dense, Activation 

model = Sequential() 
model.add( Dense(units=1, activation="sigmoid", input_dim= X_train_counts.shape[1], use_bias=False) )

Using TensorFlow backend.


In [4]:
sgd = keras.optimizers.SGD(lr=0.0001*400)
model.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy'])

In [5]:
from keras.utils import to_categorical
train_labels = to_categorical(Y_train)
test_labels = to_categorical(Y_test)
train_log = model.fit(X_train_counts, Y_train, epochs=10)

Epoch 1/10
348600/348600 [==============================] - 720s 2ms/step - loss: 0.3713 - acc: 0.8535
Epoch 2/10
348600/348600 [==============================] - 734s 2ms/step - loss: 0.3060 - acc: 0.8814
Epoch 3/10
348600/348600 [==============================] - 747s 2ms/step - loss: 0.2868 - acc: 0.8891
Epoch 4/10
348600/348600 [==============================] - 681s 2ms/step - loss: 0.2755 - acc: 0.8937
Epoch 5/10
348600/348600 [==============================] - 679s 2ms/step - loss: 0.2677 - acc: 0.8968
Epoch 6/10
348600/348600 [==============================] - 661s 2ms/step - loss: 0.2618 - acc: 0.8992
Epoch 7/10
348600/348600 [==============================] - 662s 2ms/step - loss: 0.2570 - acc: 0.9012
Epoch 8/10
348600/348600 [==============================] - 663s 2ms/step - loss: 0.2530 - acc: 0.9027
Epoch 9/10
348600/348600 [==============================] - 664s 2ms/step - loss: 0.2497 - acc: 0.9038
Epoch 10/10
348600/348600 [==============================] - 663s 2ms/ste

In [6]:
def calc_classification_error(predictions, class_labels):
    n = predictions.size
    num_of_errors = 0.
    for idx in range(n):
        if (predictions[idx] >= 0.5 and class_labels[idx]==0) or (predictions[idx] < 0.5 and class_labels[idx]==1):
            num_of_errors += 1
    return num_of_errors/n

In [7]:
predictions = model.predict(X_test_counts)
# print("Classification Error on Training Set: %.2f%%" %(calc_classification_error(predictions, class_labels) * 100))

In [8]:
print (predictions.flatten())
print (Y_test.shape)
print (np.round(0.55))

[  9.43452537e-01   2.50476114e-02   5.69929421e-01 ...,   9.28300899e-04
   3.80459398e-01   8.10950249e-03]
(149400,)
1.0


In [9]:
# for i in range(len(predictions)):
print (model.evaluate(X_test_counts, Y_test))



149400/149400 [==============================] - 277s 2ms/step
[0.2461475504242592, 0.90384872824951024]


In [56]:
# print('Accuracy with logistic regression:....', np.mean(predictions==Y_test))

Accuracy with logistic regression:.... 0.0


/Users/hamzakhan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  """Entry point for launching an IPython kernel.


In [71]:
myTweet = ['I got cock blocked', 'I have a big car']
# X_test = tweets_test[:,0]
my_test_counts = vectorizer.transform(myTweet)
predictions = model.predict(my_test_counts)

print (predictions)

[[ 0.57011467]
 [ 0.65407145]]
